In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13069,2024-03-21,Eua Nba,20:00,Orlando Magic,New Orleans Pelicans,1.88,1.78,209.5,1.84,1.96,1.5,1.93,1.74,2J1cMMPf,0.531915,0.561798,0.543478,0.510204,0.093713,0.6,0.2,NaN,NaN,173.178,22.671262,0.130913,162.618,29.482496,0.181299,185.978,200.244,177.51,186.30,0.0,0.0,0.0,0.0,0.038640,0.044659,0.073215,1.42,0.284,3.098592,0.535454,0.6,0.064546,2.33,0.466,1.673820,0.576011,0.7,0.123989
13070,2024-03-21,Eua Nba,21:00,Houston Rockets,Chicago Bulls,1.71,1.98,212.5,1.83,1.97,-3.5,2.01,2.53,boe6K0f7,0.584795,0.505051,0.546448,0.507614,0.089846,0.2,0.8,NaN,NaN,206.842,47.698151,0.230602,302.528,106.876166,0.353277,244.264,359.100,265.10,477.50,0.0,0.0,0.0,0.0,0.103479,0.052103,0.161980,1.38,0.276,2.572464,0.475659,0.5,0.024341,3.21,0.642,1.526480,0.416384,0.5,0.083616
13071,2024-03-21,Europa Euroliga,15:00,Zalgiris Kaunas,Baskonia,1.71,2.20,163.5,1.82,1.92,-2.5,1.95,2.60,0tzPjsw9,0.584795,0.454545,0.549451,0.520833,0.039341,0.8,1.0,NaN,NaN,175.880,40.820485,0.232093,196.586,91.484335,0.465365,180.832,276.106,156.48,342.72,0.0,0.0,0.0,0.0,0.177229,0.037813,0.202031,3.04,0.608,1.167763,0.525589,0.6,0.074411,-2.36,-0.472,-2.542373,0.373853,0.5,0.126147
13072,2024-03-21,Europa Euroliga,16:30,Partizan,Real Madrid,2.05,1.80,167.5,1.91,1.91,1.5,1.91,1.63,KUncYYMs,0.487805,0.555556,0.523560,0.523560,0.043360,0.2,0.4,NaN,NaN,127.408,48.538835,0.380972,119.648,20.119407,0.168155,105.060,131.796,102.18,144.18,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,-1.16,-0.232,-4.525862,0.849711,0.8,-0.049711,-1.97,-0.394,-2.030457,0.632175,0.6,-0.032175
13073,2024-03-21,Argentina Liga A,21:00,Union De Santa Fe,La Union,1.90,1.82,161.5,1.85,1.85,3.5,1.55,1.48,445p8Qf6,0.526316,0.549451,0.540541,0.540541,0.075766,0.8,0.6,NaN,NaN,232.184,53.121863,0.228792,283.366,95.710166,0.337762,211.250,264.738,266.00,346.11,0.0,0.0,0.0,0.0,0.030413,0.000000,0.032672,0.44,0.088,10.227273,0.000000,0.0,0.000000,-5.00,-1.000,-0.820000,0.000000,0.0,0.000000
13074,2024-03-21,Áustria Superliga,15:00,Kapfenberg,Vienna,2.26,1.59,169.5,1.81,1.89,2.5,1.99,1.50,WjopzbSu,0.442478,0.628931,0.552486,0.529101,0.071409,0.4,0.6,NaN,NaN,164.286,60.971435,0.371130,126.754,65.840626,0.519436,121.900,132.326,116.27,92.65,0.0,0.0,0.0,0.0,0.246110,0.030578,0.198557,-3.96,-0.792,-1.590909,0.000000,0.0,0.000000,-2.88,-0.576,-1.024306,0.000000,0.0,0.000000
13075,2024-03-21,China Cba,08:35,Jilin,Shanxi Zhongyu,2.27,1.60,225.5,1.81,1.93,2.5,1.99,1.39,dUSY6Ze4,0.440529,0.625000,0.552486,0.518135,0.065529,0.8,0.4,NaN,NaN,211.830,71.777425,0.338844,283.692,110.605896,0.389880,268.464,222.228,241.00,248.46,0.0,0.0,0.0,0.0,0.244838,0.045376,0.251044,-3.27,-0.654,-1.941896,0.412299,0.5,0.087701,-3.80,-0.760,-0.789474,0.463961,0.2,-0.263961
13076,2024-03-21,Coreia Do Sul Kbl,07:00,Anyang JungKwanJang,KoGas,2.30,1.61,165.5,1.86,1.88,2.5,1.94,1.38,KCHeVsCP,0.434783,0.621118,0.537634,0.531915,0.055901,0.6,0.2,NaN,NaN,133.758,32.096435,0.239959,162.044,21.159307,0.130578,181.926,172.354,127.65,137.70,0.0,0.0,0.0,0.0,0.249567,0.007563,0.238542,-3.04,-0.608,-2.138158,0.402179,0.4,-0.002179,-1.28,-0.256,-2.382813,0.410376,0.6,0.189624
13077,2024-03-21,Coreia Do Sul Kbl,07:00,Seoul Knights,Wonju DB,1.64,2.25,165.5,1.83,1.91,-3.5,1.94,2.90,ML3NQ1Kt,0.609756,0.444444,0.546448,0.523560,0.054201,0.4,0.4,NaN,NaN,148.380,35.102404,0.236571,129.450,16.196663,0.125119,127.620,132.370,153.90,157.53,0.0,0.0,0.0,0.0,0.221766,0.030251,0.280505,-1.94,-0.38

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Houston Rockets,Chicago Bulls,212.5,1.83,1.0000,Over
1,15:00,Europa Euroliga,Zalgiris Kaunas,Baskonia,163.5,1.82,1.0000,Over
2,07:00,Coreia Do Sul Kbl,Seoul Knights,Wonju DB,165.5,1.83,1.0000,Over
3,13:40,Eua Ncaa,Brigham Young,Duquesne,141.5,1.80,1.0000,Over
4,14:30,Eua Ncaa,Creighton,Akron,140.5,1.80,1.0000,Over
5,16:10,Eua Ncaa,Illinois,Morehead State,146.5,1.80,1.0000,Over
6,17:30,Eua Ncaa,Dayton,Nevada,136.5,1.91,1.0000,Over
7,13:30,Polônia Liga De Basquete,Lancut,Torun,163.5,1.80,0.9940,Over
8,21:00,Eua Nba,Milwaukee Bucks,Brooklyn Nets,222.5,1.93,1.0000,Over
9,23:30,Eua Nba,Phoenix Suns,Atlanta Hawks,226.5,1.93,1.0000,Over
